In [ ]:
import os
import datadbhandler as dbhandler
import pandas as pd
import numpy as np

PATH = 'D:/Trading/Historical Data/'
CSVPATH = 'NSE EOD/'
DBPATH = 'db/nsedb.db'
MOD_PATH = 'nse_eod_modifiers/'
AMI_PATH = 'amibroker_files/'

os.chdir(PATH)

db = dbhandler.DataDB(DBPATH)

In [ ]:
df = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)

In [ ]:
df.head()

In [ ]:
df.iloc[0][1]

In [ ]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=df.iloc[0][1], end_date=df.iloc[0][1])

In [ ]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=19960918, end_date=19960918)

In [ ]:
db.truncate_table('tblDumpReplace', msg=True)

In [ ]:
db.truncate_table('tblMultipliers', msg=True)

In [ ]:
db.truncate_table('tblSkipped', msg=True)

In [ ]:
db.truncate_table('tblSymbolRange', msg=True)

In [ ]:
db.truncate_table('tblHistIndex', msg=True)

In [ ]:
db.truncate_table('tblDuplicates', msg=True)

In [ ]:
for i in range(1995, 2020):
    db.truncate_table('tblModDump{}'.format(i), msg=True)

In [ ]:
import pandas as pd

In [ ]:
df = dbhandler.pd.read_csv(db.SYMBOL_MAPPING_FILE)

In [ ]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

In [ ]:
import numpy as np
manual = dbhandler.pd.read_csv(db.INDEX_CHANGE_MANUAL_CSV, encoding='ansi')

for symbol in ['BAYERCROP', 'ABBOTINDIA']: #manual.Symbol.unique():
    symbol_dates = manual[manual.Symbol == symbol]['Date'].unique()
    #print(1, symbol, symbol_dates)
    #print(2, manual[manual.Symbol == symbol].CorrectSymbol)
    correct_symbol = manual[manual.Symbol == symbol].CorrectSymbol.iloc[0]
    #print(3, symbol, correct_symbol, symbol_dates)
    for date in symbol_dates:
        #print(4, symbol, date, correct_symbol)
        #print(5, np.where((df2.Symbol == symbol) & (df2.Date == date), correct_symbol, df2.Symbol))
        #print(5, np.where(df2.Date == date, correct_symbol, df2.Symbol))
        #print(0, symbol, date, correct_symbol)
        #print(1, df2[df2.Symbol == symbol])
        #print(2, df2[df2.Date == date])
        #print(3, df2[(df2.Symbol == symbol) & (df2.Date == date)])
        df2['Symbol'] = np.where((df2.Symbol == symbol) & (df2.Date == int(date)), correct_symbol, df2.Symbol)
df2.to_csv(db.INDEX_CHANGE_MOD_CSV)

In [ ]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

In [ ]:
df

In [ ]:
import pandas as pd

In [ ]:
symbols_range = pd.read_sql_query('''SELECT Symbol, MIN(Date) MinDate, MAX(Date) MaxDate 
                                        FROM tblModDump{} WHERE Date >= {} 
                                        GROUP BY Symbol'''.format(2020, 20200805), db.conn)

In [ ]:
symbols_range.index = symbols_range.Symbol
symbols_range

In [ ]:
start_date = '20200805'
qry = '''SELECT Symbol, Min(Date) MinDate, Max(Date) MaxDate, Source, TableSource, StartDate, EndDate
                FROM (SELECT t1.Symbol, t1.Date, 'Dump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		AND TableSource <> 'ModDump'
		WHERE t1.Date >= {start_date}
		UNION
	SELECT t1.Symbol, t1.Date, 'ModDump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblModDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		AND TableSource <> 'Dump'
		WHERE t1.Date >= {start_date}
		UNION
        SELECT t1.Symbol, t1.Date, 'Dump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		WHERE t1.Date >= {start_date}
		UNION
	SELECT t1.Symbol, t1.Date, 'ModDump' Source, t2.TableSource, t2.StartDate, t2.EndDate
		FROM tblModDump{year} t1
		LEFT JOIN tblSymbolRange t2
		ON t1.Symbol = t2.Symbol
		WHERE t1.Date >= {start_date}) t1
        WHERE Source = TableSource 
        OR TableSource IS NULL
        GROUP BY Symbol, Source, TableSource'''.format(start_date=start_date, year=start_date[0:4])

In [ ]:
join = pd.read_sql_query(qry, db.conn)

In [ ]:
join

In [1]:
import history
PATH = 'D:/Trading/Historical Data/'
DBPATH = 'db/nsedb.db'

os.chdir(PATH)

In [3]:
hist = history.History(DBPATH)

Opening Bhavcopy database db/nsedb.db...


In [ ]:
data = hist.symbol_history('RELCAPITAL', start_date='19981001', end_date='20200801', buffer_start='19980801')

In [ ]:
df1, df2 = hist.index_change_history('Nifty 50')

In [ ]:
df1.head(5)

In [ ]:
df2.head(5)

In [ ]:
dfx = df1[df1.Symbol == 'RELCAPITAL'] 

In [ ]:
dfx

In [ ]:
import pandas as pd
import numpy as np
df3 = pd.merge(df2, dfx, how='left', on=['Date']).sort_values(by='Date')

In [ ]:
df3

In [ ]:
df3.Symbol = np.where(df3.IndexName.notnull(), True, np.nan)

In [ ]:
df3.Symbol.fillna(method='ffill', inplace=True, limit=1)

In [ ]:
df3

In [ ]:
df3.Symbol = np.where((df3.IndexName.isnull()) & (df3.Symbol.notnull()), False, df3.Symbol)

In [ ]:
df3

In [ ]:
df3 = df3[df3.Symbol.notnull()][['Date', 'Symbol']]

In [ ]:
df3.columns = ['Date', 'IndexFlag']

In [ ]:
df3

In [ ]:
df4 = pd.merge(data, df3, how='left', on=['Date']).sort_values(by='Date')

In [ ]:
df4.iloc[150:200]

In [ ]:
df4[df4.IndexFlag.notnull()]

In [ ]:
df5 = df4

In [ ]:
df5 = df5.IndexFlag.fillna(method='ffill')

In [ ]:
df5.IndexFlag = np.where(df5.IndexFlag is False, np.nan, df5.IndexFlag)

In [ ]:
df5.iloc[3850:3900]

In [ ]:
df5.head(50)

In [ ]:
df3[df3.Date.astype(int) < 19980801].sort_values(by='Date', ascending=False).iloc[0].IndexFlag

In [ ]:
dfn = hist.symbol_history_index_flagged('RELCAPITAL', 'Nifty 50', start_date='19981001', end_date='20200801', buffer_start='19980801')

In [ ]:
dfn

In [ ]:
df5.IndexFlag[45:].equals(dfn.IndexFlag[45:])

In [ ]:
df5[45:].equals(dfn[45:])

In [ ]:
for col in df5.columns:
    print(col, df5[col].equals(dfn[col]))

In [ ]:
df5.Buffer.sum(), dfn.Buffer.sum()

In [ ]:
df5.IndexFlag.sum(), dfn.IndexFlag.sum()

In [ ]:
df5.IndexFlag[:44] == dfn.IndexFlag[:44]

In [ ]:
dfn[['Date', 'IndexFlag']][3250:3300]

In [ ]:
dfn

In [4]:
data = hist.symbol_history('RELCAPITAL', start_date='19981001', end_date='19970801', buffer_start='19980801')

AssertionError: End date 19970801 cannot be less than start date 19981001

In [5]:
hist.index_components_history('Nifty 50', start_date='19981001', end_date='19970801', buffer_start='19980801')

AssertionError: End date 19970801 cannot be less than start date 19981001

In [ ]:
data = hist.symbol_history('RELCAPITAL', start_date='19981001', 'Nifty 50', end_date='19970801', buffer_start='19980801')